unzip data

https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data

In [1]:
!file /content/all_data.csv.zip

/content/all_data.csv.zip: Zip archive data, at least v4.5 to extract, compression method=deflate


In [2]:
!ls -lh /content/all_data.csv.zip


-rw-r--r-- 1 root root 327M Jan 12 09:06 /content/all_data.csv.zip


In [3]:
!7z x /content/all_data.csv.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 341878979 bytes (327 MiB)

Extracting archive: /content/all_data.csv.zip
--
Path = /content/all_data.csv.zip
Type = zip
Physical Size = 341878979

  0%      1% - all_data.csv                     4% - all_data.csv                     6% - all_data.csv                     8% - all_data.csv                    10% - all_data.csv                    12% - all_data.csv                    13% - all_data.csv                   

let's explore the data

In [4]:
pip install num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d6f579740c9d710e0502f26eff3e6072ba19ccd1bc8d86bea2b7de43a8a0d3df
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [72]:
import pandas as pd
import numpy as np
import re
import html
from num2words import num2words
import string
import unicodedata
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
df = pd.read_csv('/content/all_data.csv')

In [7]:
df.head(5)

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [9]:
df.columns

Index(['id', 'comment_text', 'split', 'created_date', 'publication_id',
       'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes',
       'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'identity_annotator_count',
       'toxicity_annotator_count'],
      dtype='object')

In [42]:
unique_values,count = np.unique(df['split'], return_counts=True)
print(unique_values)
print(count)

['test' 'train']
[ 194641 1804875]


In [43]:
df['comment_text'][0]

"He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed."

In [44]:
unique_values,count = np.unique(df['rating'], return_counts=True)
print(unique_values)
print(count)

['approved' 'rejected']
[1867134  132382]


lets do some operation on data before text preprocessing

In [45]:
train_texts = df[df['split'] == 'train']['comment_text'].values
test_texts = df[df['split'] == 'test']['comment_text'].values

train_labels = df[df['split'] == 'train']['toxicity'].values
test_labes = df[df['split'] == 'test']['toxicity'].values

In [46]:
train_labels = np.asarray(train_label>=0.5).astype('float32')
test_labels = np.asarray(test_label>=0.5).astype('float32')

In [47]:
train_texts[0]

"He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed."

In [48]:
train_texts.shape

(1804875,)

In [49]:
train_labels[0]

np.float32(0.0)

In [50]:
train_labels.shape

(1804875,)

# text preprocessing

In [79]:
def remove_specific_chars(text):
  re1 = re.compile(r'  +')
  x = text.replace('#39;', "").replace('amp;', '').replace('#146;', "").replace(
      'nbsp;', '').replace('#36;', '').replace('\\n', "").replace('quot;', "").replace(
      '<br />', "").replace('\\"', '').replace('<unk>','').replace(' @.@ ','').replace(
      ' @-@ ','').replace('\\', '')
  return re1.sub(' ',html.unescape(x))


def remove_non_ascii(text):
  return unicodedata.normalize('NFKD' ,text).encode('ascii' ,'ignore').decode('utf8' ,'ignore')


def to_lowercase(text):
  return text.lower()


def remove_punctuation(text):
  translator = str.maketrans('','',string.punctuation)
  return text.translate(translator)


def replace_numbers(text):
  return re.sub(r'\d+' , lambda x: num2words(int(x.group())) ,text)


def remove_whitespaces(text):
  return text.strip()


def remove_stopwords(words,stop_words):
  return [word for word in words if word not in stop_words]


def lemmatizer_verbs(words):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(w , pos = 'v') for w in words]


def text2words(text):
  return word_tokenize(text)


def normalize_text(text):

  if not isinstance(text,str):
      return ""
  stop_words = stopwords.words('english')
  text = remove_specific_chars(text)
  text = remove_non_ascii(text)
  text = remove_punctuation(text)
  text = to_lowercase(text)
  text = replace_numbers(text)
  text = remove_whitespaces(text)

  words = text2words(text)
  words = remove_stopwords(words,stop_words)
  words = lemmatizer_verbs(words)

  return ' '.join(words)


Now let's apply this on the whole corpus

In [80]:
def normlize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [ ]:
norm_train_texts =  normlize_corpus(train_text)
norm_test_texts =  normlize_corpus(test_text)

In [ ]:
norm_train_texts[0]

# text preperation
we will use keras tokenizer

In [ ]:
from tensorflow.keras.preproxessing.text import Tokenizer

In [ ]:
text = norm_train_texts+norm_test_texts

In [ ]:
tok = Tokenizer(oov_token = 'UNK' , num_words = 50000)
tok.fit_on_texts(text)
